In [1]:
# Import packages
import geopandas as gpd
import numpy as np
import pandas as pd
import libpysal
import networkx as nx
import osmnx as ox
import time
from shapely import geometry
from shapely.geometry import Point, MultiLineString, LineString, Polygon
from shapely.ops import nearest_points
import matplotlib.pyplot as plt
from itertools import product, combinations
import multiprocessing as mp
import math
import warnings
import igraph

In [2]:
# Block 0 cities and assumptions

start = time.time()

cities = ['Philadelphia, United States','Denver, United States','Ghent, Belgium','Amsterdam, Netherlands',
          'Dhaka Metropolitan, Bangladesh']

# Assumptions
thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m
walkable_park_dist = 500 # radius in metres
one_park_buffer = 25 # in metres
park_entry_point_buffer = 25 # in metres
min_park_size = 400 # in squared metres (WHO = 0.04 ha = 400m)
park_entry_point_merge = 0 # (default)

thresholds
# 'Dublin, Ireland' is problematic; road system is too dense, has separate script

[300, 600, 1000]

In [3]:
# Block 1 city boundaries

warnings.filterwarnings("ignore")
bound_df = pd.DataFrame()
for i in range(len(cities)):
    bound_df = pd.concat([bound_df, pd.DataFrame(ox.geocoder.geocode_to_gdf(cities[i]))])
    
bound_df = bound_df.reset_index()
bound_df = bound_df.loc[:,bound_df.columns!='index']
bound_df = gpd.GeoDataFrame(bound_df, geometry = 'geometry', crs = 4326)
bound_df.to_file(r'C:\Users\bartb\Downloads\bounds.shp')
bound_df

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((-75.28030 39.97500, -75.28022 39.974...",40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,40.002410,-75.139364,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797
1,"POLYGON ((-105.10988 39.62710, -105.10761 39.6...",39.914209,39.614315,-104.599700,-105.109885,327393456,relation,1411339,39.739236,-104.984862,"Denver, Colorado, United States",boundary,administrative,0.905476
2,"POLYGON ((3.57976 51.03027, 3.57992 51.03022, ...",51.188886,50.979542,3.849325,3.579762,282032351,relation,897671,51.084199,3.732851,"Ghent, Gent, East Flanders, Flanders, Belgium",boundary,administrative,0.855558
3,"MULTIPOLYGON (((4.72876 52.40071, 4.73371 52.4...",52.431064,52.278174,5.079162,4.728756,282057196,relation,271110,52.372760,4.893604,"Amsterdam, North Holland, Netherlands",boundary,administrative,0.946813
4,"POLYGON ((90.32978 23.75081, 90.32981 23.75050...",23.899531,23.668077,90.508832,90.329784,328693247,relation,13663697,23.783663,90.403246,"Dhaka Metropolitan, Dhaka District, Dhaka Divi...",boundary,administrative,0.710000


In [4]:
# Block 2 Road networks

warnings.filterwarnings("ignore")

start_time = time.time()
graphs = list()
road_nodes = list()
road_edges = list()
road_conn = list()
roads = list()

for i in range(len(cities)):
    # Get graph, road nodes and edges
    graph = ox.graph_from_place(cities[i], network_type="all", buffer_dist = (np.max(thresholds)+1000))
    graphs.append(graph)
    road_node, road_edge = ox.graph_to_gdfs(graph)
    
    # Road nodes format
    road_node = road_node.to_crs(4326)
    road_node['geometry_m'] = gpd.GeoSeries(road_node['geometry'], crs = 4326).to_crs(3043)
    road_nodes.append(road_node)
    road = road_node.reset_index()
    roads.append(road)
    
    # format road edges
    road_edge = road_edge.to_crs(4326)
    road_edge['geometry_m'] = gpd.GeoSeries(road_edge['geometry'], crs = 4326).to_crs(3043)
    road_edge = road_edge.reset_index()
    road_edge.rename(columns={'u':'from', 'v':'to'}, inplace=True)
    road_edge['key'] = road_edge['from'].astype(str) + '-' + road_edge['to'].astype(str)
    road_edges.append(road_edge)
    
    # Get only necessary road connections columns for network performance
    road_con = road_edge[['osmid','key','length','geometry']]
    road_con = road_con.set_index('key')
    road_conn.append(road_con)
    print(cities[i].rsplit(',')[0], 'done', round((time.time() - start_time) / 60,2),'mns')
road_edges[0]

Philadelphia done 2.18 mns
Denver done 5.46 mns
Ghent done 6.13 mns
Amsterdam done 7.64 mns
Dhaka Metropolitan done 8.43 mns


,from,to,key,osmid,name,highway,oneway,length,geometry,ref,lanes,width,bridge,service,tunnel,maxspeed,access,junction,area,geometry_m
0,103237949,103353127,103237949-103353127,11591597,New Jersey Avenue,residential,False,111.799,"LINESTRING (-74.96444 40.03625, -74.96557 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689489.546 8448457.709, -5689617..."
1,103237949,103353090,103237949-103353090,11591597,New Jersey Avenue,residential,False,107.489,"LINESTRING (-74.96444 40.03625, -74.96337 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689489.546 8448457.709, -5689364..."
2,103237949,103237976,103237949-103237976,11580386,Cleveland Avenue,residential,False,168.717,"LINESTRING (-74.96444 40.03625, -74.96404 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689489.546 8448457.709, -5689552..."
3,103237976,103238007,103237976-103238007,11580386,Cleveland Avenue,residential,False,151.349,"LINESTRING (-74.96340 40.03496, -74.96245 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689653.040 8448263.511, -5689797..."
4,103237976,103590312,103237976-103590312,11610261,2nd Street,residential,False,108.508,"LINESTRING (-74.96340 40.03496, -74.96448 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689653.040 8448263.511, -5689779..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338097,9726316545,5550941352,9726316545-5550941352,579660432,NaN,service,False,39.965,"LINESTRING (-75.24128 40.06507, -75.24111 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5695926.465 8483769.053, -5695935..."
338098,9726316545,9726316548,9726316545-9726316548,1058487388,NaN,service,False,58.733,"LINESTRING (-75.24128 40.06507, -75.24148 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5695926.465 8483769.053, -5695962..."
338099,9726316548,9726316545,9726316548-9726316545,1058487388,NaN,service,False,58.733,"LINESTRING (-75.24175 40.06470, -75.24175 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5696003.982 8483808.258, -5695998..."
338100,9728655691,110122187,9728655691-110122187,12149073,South 10th Street,residential,True,3.250,"LINESTRING (-75.15829 39.94437, -75.15830 39.9...",NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5711864.888 8467388.116, -5711869..."


In [5]:
# Block 3 population grids

grids = list()
for i in range(len(cities)):
    # extract grids
    path = 'D:/Dumps/Population_grids/' + cities[i].rsplit(',')[0] + '/' + cities[i].rsplit(',')[0] + '_CPoPGrid.gpkg'
    file = gpd.read_file(path).to_crs(4326)

    # Get grid and boundary overlay
    popgrid = file.overlay(bound_df.iloc[i:i+1])

    # Only get full grids (area differs slightly due to lon-lat placement) and grids with people in it.
    popgrid['area'] = popgrid.area / popgrid.area.max()
    popgrid = popgrid[(popgrid['area'] >= 0.99)]
    popgrid = popgrid[popgrid['PoP2015_Number'] > 0]
    
    # Get grids centroids and lon-lat
    popgrid['centroid'] = popgrid.to_crs(4326).centroid
    popgrid['centroid_m'] = popgrid['centroid'].to_crs(3043)
    popgrid['grid_lon'] = popgrid['centroid_m'].x
    popgrid['grid_lat'] = popgrid['centroid_m'].y
    
    popgrid = popgrid.reset_index()
    grids.append(popgrid)
grids[0]

,index,grid_id,PoP2015_Number,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,...,display_name,class,type,importance,geometry,area,centroid,centroid_m,grid_lon,grid_lat
0,6,259,113,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.01452 40.13521, -75.01452 40.133...",0.996208,POINT (-75.01564 40.13435),POINT (-5675978.406 8459812.552),-5.675978e+06,8.459813e+06
1,7,260,135,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.01227 40.13521, -75.01227 40.133...",0.996208,POINT (-75.01339 40.13435),POINT (-5675888.831 8459539.227),-5.675889e+06,8.459539e+06
2,13,400,137,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.01676 40.13349, -75.01676 40.131...",0.996234,POINT (-75.01789 40.13262),POINT (-5676341.311 8459996.328),-5.676341e+06,8.459996e+06
3,14,401,104,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.01452 40.13349, -75.01452 40.131...",0.996234,POINT (-75.01564 40.13262),POINT (-5676251.742 8459722.980),-5.676252e+06,8.459723e+06
4,15,402,116,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.01227 40.13349, -75.01227 40.131...",0.996234,POINT (-75.01339 40.13262),POINT (-5676162.159 8459449.641),-5.676162e+06,8.459450e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7868,10279,21026,22,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.25482 39.88135, -75.25482 39.879...",0.999949,POINT (-75.25594 39.88048),POINT (-5725941.707 8476084.513),-5.725942e+06,8.476085e+06
7869,10280,21027,22,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.25257 39.88135, -75.25257 39.879...",0.999949,POINT (-75.25369 39.88048),POINT (-5725852.550 8475808.155),-5.725853e+06,8.475808e+06
7870,10281,21028,2,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.25033 39.88135, -75.25033 39.879...",0.999949,POINT (-75.25145 39.88048),POINT (-5725763.377 8475531.806),-5.725763e+06,8.475532e+06
7871,10313,21166,6,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.25931 39.87962, -75.25931 39.877...",0.999975,POINT (-75.26043 39.87875),POINT (-5726396.363 8476548.128),-5.726396e+06,8.476548e+06


In [6]:
# Block 4 city greenspace

parks_in_range = list()
for i in range(len(cities)):
    path = 'D:/Dumps/Greenspace/' + cities[i].rsplit(',')[0] + '/' + cities[i].rsplit(',')[0] + '_Greenspace.gpkg'
    greenspace = gpd.read_file(path).to_crs(4326)

    warnings.filterwarnings("ignore")

    green_buffer = gpd.GeoDataFrame(geometry = greenspace.to_crs(3043).buffer(one_park_buffer).to_crs(4326))
    greenspace['geometry_w_buffer'] = green_buffer
    greenspace['geometry_w_buffer'] = gpd.GeoSeries(greenspace['geometry_w_buffer'], crs = 4326)
    greenspace['geom buffer diff'] = greenspace['geometry_w_buffer'].difference(greenspace['geometry'])

    # This function group components in itself that overlap (with the buffer set of 25 metres)
    # https://stackoverflow.com/questions/68036051/geopandas-self-intersection-grouping
    W = libpysal.weights.fuzzy_contiguity(greenspace['geometry_w_buffer'])
    greenspace['components'] = W.component_labels
    parks = greenspace.dissolve('components')

    # Exclude parks below 0.04 ha.
    parks = parks[parks.to_crs(3043).area > min_park_size]
        
    # Get the park buffer
    parks_t = pd.DataFrame(parks.index).set_index('components')
    parks_t['park_geom'] = parks['geometry']
    parks_t = gpd.GeoDataFrame(parks_t, geometry = 'park_geom', crs = 4326)
    for j in range(len(thresholds)):
        Bound_buffer = gpd.GeoDataFrame(
            geometry = np.repeat(bound_df.iloc[i:i+1].to_crs(3043).
                                 buffer(thresholds[j]).to_crs(4326),len(parks)), 
            crs = 4326).reset_index().iloc[:,1]
        Bound_buffer.index = parks.index
        parks_t = parks_t.join(Bound_buffer, how = 'outer')
        parks_t = parks_t.rename(columns = {'geometry':('buffer_' + str(thresholds[j]))})
        
        # Check which parks intersect with the city boundary with buffer.
        parks_within_range = parks_t[parks_t['park_geom'].intersection(parks_t[('buffer_' + str(thresholds[j]))]).area > 0]
        pwr = pd.DataFrame(parks_within_range.index).set_index('components')
        pwr[str(thresholds[j])] = 1
        parks_t = parks_t.join(pwr, how = 'left')
        parks_t[str(thresholds[j])] = parks_t[str(thresholds[j])].fillna(0)
    parks_t = parks_t.join(parks)
    
    thresholds_str = list()
    for k in range(len(thresholds)): thresholds_str.append(str(thresholds[k]))
    parks_t = parks_t[(parks_t[thresholds_str] == 1).any(axis = 1)]
    parks_t = parks_t.reset_index()
    parks_t['geometry_m'] = parks_t['geometry'].to_crs(3043)
    parks_in_range.append(parks_t)
    
parks_in_range[0]

,components,park_geom,buffer_300,300,buffer_600,600,buffer_1000,1000,geometry,OBJECTID,...,IUCNCtDt,Date_Est,Comments,EsmtHldr,EHoldTyp,SHAPE_Leng,SHAPE_Area,geometry_w_buffer,geom buffer diff,geometry_m
0,5,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.24927 39.87455, -75.24988 39.874...",1.0,"POLYGON ((-75.25010 39.87261, -75.25090 39.872...",1.0,"POLYGON ((-75.25412 39.87008, -75.25420 39.870...",1.0,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",81288,...,2020,1983,None,Natural Lands Trust,NGO,2167.370365,1.070843e+05,"POLYGON ((-75.25609 40.06118, -75.25608 40.061...","POLYGON ((-75.25608 40.06117, -75.25606 40.061...","MULTIPOLYGON (((-5698736.789 8484341.822, -569..."
1,6,"POLYGON ((-75.24702 39.91695, -75.24706 39.916...","POLYGON ((-75.24927 39.87455, -75.24988 39.874...",1.0,"POLYGON ((-75.25010 39.87261, -75.25090 39.872...",1.0,"POLYGON ((-75.25412 39.87008, -75.25420 39.870...",1.0,"POLYGON ((-75.24702 39.91695, -75.24706 39.916...",82731,...,2020,None,None,None,None,36128.050943,3.353065e+06,"POLYGON ((-75.24691 39.91683, -75.24696 39.916...","POLYGON ((-75.24696 39.91680, -75.24697 39.916...","POLYGON ((-5719765.167 8476866.315, -5719770.3..."
2,7,"MULTIPOLYGON (((-75.25450 39.95036, -75.25451 ...","POLYGON ((-75.24927 39.87455, -75.24988 39.874...",1.0,"POLYGON ((-75.25010 39.87261, -75.25090 39.872...",1.0,"POLYGON ((-75.25412 39.87008, -75.25420 39.870...",1.0,"MULTIPOLYGON (((-75.25450 39.95036, -75.25451 ...",82731,...,2020,None,None,None,None,36128.050943,3.353065e+06,"POLYGON ((-75.24828 39.92126, -75.24836 39.921...","POLYGON ((-75.24836 39.92121, -75.24837 39.921...","MULTIPOLYGON (((-5714730.553 8479503.851, -571..."
3,8,"POLYGON ((-75.14985 39.96195, -75.14993 39.961...","POLYGON ((-75.24927 39.87455, -75.24988 39.874...",1.0,"POLYGON ((-75.25010 39.87261, -75.25090 39.872...",1.0,"POLYGON ((-75.25412 39.87008, -75.25420 39.870...",1.0,"POLYGON ((-75.14985 39.96195, -75.14993 39.961...",1260,...,2020,1978,Edgar Allan Poe National Historic Site,None,None,216.240707,2.114309e+03,"POLYGON ((-75.15037 39.96185, -75.15037 39.961...","POLYGON ((-75.15037 39.96186, -75.15038 39.961...","POLYGON ((-5708727.262 8467262.277, -5708788.5..."
4,9,"MULTIPOLYGON (((-75.17212 40.03366, -75.17215 ...","POLYGON ((-75.24927 39.87455, -75.24988 39.874...",1.0,"POLYGON ((-75.25010 39.87261, -75.25090 39.872...",1.0,"POLYGON ((-75.25412 39.87008, -75.25420 39.870...",1.0,"MULTIPOLYGON (((-75.17212 40.03366, -75.17215 ...",1348,...,2020,None,Independence National Historical Park,None,None,6162.483495,1.285207e+05,"POLYGON ((-75.17280 40.03368, -75.17280 40.033...","POLYGON ((-75.17280 40.03369, -75.17280 40.033...","MULTIPOLYGON (((-5698195.124 8473695.244, -569..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,734,"POLYGON ((-75.25746 39.94945, -75.25807 39.949...","POLYGON ((-75.24927 39.87455, -75.24988 39.874...",0.0,"POLYGON ((-75.25010 39.87261, -75.25090 39.872...",0.0,"POLYGON ((-75.25412 39.87008, -75.25420 39.870...",1.0,"POLYGON ((-75.25746 39.94945, -75.25807 39.949...",242021,...,2020,None,None,None,None,217.559826,1.590417e+03,"POLYGON ((-75.25744 39.94930, -75.25744 39.949...","POLYGON ((-75.25744 39.94930, -75.25806 39.949...","POLYGON ((-5714991.814 8479820.456, -5715022.3..."
469,735,"POLYGON ((-75.27556 39.96885, -75.27571 39.968...","POLYGON ((-75.24927 39.87455, -75.24988 39.874...",1.0,"POLYGON ((-75.25010 39.87261, -75.25090 39.872...",1.0,"POLYGON ((-75.25412 39.87008, -75.25420 39.870...",1.0,"POLYGON ((-75.27556 39.96885, -75.27571 39.968...",242022,...,2020,None,None,None,None,315.117880,4.314566e+03,"POLYGON ((-75.27542 39.96896, -75.27542 39.968...","POLYGON ((-75.27542 39.96896, -75.27541 39.968...","POLYGON ((-5712611.388 8483038.861, -5712651.7..."
470,736,"POLYGON ((-75.25665 39.94613, -75.25671 39.946...","POLYGON ((-75.24927 39.87455, -75.24988 39.874...",0.0,"POLYGON ((-75.25010 39.87261, -75.25090 39.872...",1.0,"POL

In [7]:
# Block 5 park entry points

start_time = time.time()
ParkRoads = list()
for j in range(len(cities)):
    ParkRoad = pd.DataFrame()
    mat = list()
    # For all
    for i in range(len(parks_in_range[j])):
        dist = road_nodes[j]['geometry'].to_crs(3043).distance(parks_in_range[j]['geometry'].to_crs(
            3043)[i])
        buf_nodes = road_nodes[j][(dist < park_entry_point_buffer) & (dist > 0)]
        mat.append(list(np.repeat(i, len(buf_nodes))))
        ParkRoad = pd.concat([ParkRoad, buf_nodes])
        if i % 50 == 0: print(cities[j].rsplit(',')[0], round(i/len(parks_in_range[j])*100,1),'% done', 
                              round((time.time() - start_time) / 60,2),' mns')
    # Park no list conversion
    mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat) for i in b]

    # Format
    ParkRoad['Park_No'] = mat_u
    ParkRoad = ParkRoad.reset_index()
    ParkRoad['park_lon'] = ParkRoad['geometry_m'].x
    ParkRoad['park_lat'] = ParkRoad['geometry_m'].y
    
    # Get the road nodes intersecting with the parks' buffer
    ParkRoad = pd.merge(ParkRoad, parks_in_range[j][['park_geom','geometry']], left_on = 'Park_No', right_index = True)
    ParkRoad = pd.merge(ParkRoad, parks_in_range[j][thresholds_str], left_on = 'Park_No', right_index = True)
    
    # Get the walkable park size
    ParkRoad['park_size_walkable'] = ParkRoad['geometry_m'].buffer(walkable_park_dist).to_crs(4326).intersection(ParkRoad['geometry_y'])
    ParkRoad['walk_area'] = ParkRoad['park_size_walkable'].to_crs(3043).area
    ParkRoad['park_area'] = ParkRoad['geometry_y'].to_crs(3043).area
    ParkRoad['share_walked'] = ParkRoad['walk_area'] / ParkRoad['park_area']
    
    # Get size inflation factors for the gravity model
    ParkRoad['size_infl_factor'] = ParkRoad['walk_area'] / ParkRoad['walk_area'].median()
    ParkRoad['size_infl_proot2'] = ParkRoad['size_infl_factor']**(1/2)
    ParkRoad['size_infl_proot3'] = ParkRoad['size_infl_factor']**(1/3)
    ParkRoad['size_infl_proot5'] = ParkRoad['size_infl_factor']**(1/5)
    ParkRoads.append(ParkRoad)
    
    fltr = ParkRoad.loc[:,~ParkRoad.columns.isin(['geometry_x','geometry_m','geometry_y','park_size_walkable', 
                                                          'geometry_m_buffer'])]
                     
    gdf = gpd.GeoDataFrame(pd.DataFrame(fltr), geometry = 'park_geom', crs = 4326)
    gdf.to_file('D:Dumps/Scores output/Park_entry_points/'+ cities[j] +'.shp')
    
    print(cities[j].rsplit(',')[0],'100 % done', 
                              round((time.time() - start_time) / 60,2),' mns')
    
ParkRoads[0]

Philadelphia 0.0 % done 0.01  mns
Philadelphia 10.6 % done 1.01  mns
Philadelphia 21.1 % done 1.76  mns
Philadelphia 31.7 % done 2.1  mns
Philadelphia 42.3 % done 2.48  mns
Philadelphia 52.9 % done 2.81  mns
Philadelphia 63.4 % done 3.11  mns
Philadelphia 74.0 % done 3.49  mns
Philadelphia 84.6 % done 3.86  mns
Philadelphia 95.1 % done 4.27  mns
Denver 0.0 % done 5.54  mns
Denver 14.7 % done 9.06  mns
Denver 29.3 % done 10.56  mns
Denver 44.0 % done 12.78  mns
Denver 58.7 % done 14.97  mns
Denver 73.3 % done 15.57  mns
Denver 88.0 % done 16.15  mns
Ghent 0.0 % done 16.96  mns
Ghent 40.3 % done 17.13  mns
Ghent 80.6 % done 17.31  mns
Amsterdam 0.0 % done 17.46  mns
Amsterdam 64.9 % done 17.8  mns
Dhaka Metropolitan 0.0 % done 18.01  mns
Dhaka Metropolitan 29.8 % done 18.12  mns
Dhaka Metropolitan 59.5 % done 18.22  mns
Dhaka Metropolitan 89.3 % done 18.32  mns


,osmid,y,x,street_count,highway,ref,geometry_x,geometry_m,Park_No,park_lon,...,600,1000,park_size_walkable,walk_area,park_area,share_walked,size_infl_factor,size_infl_proot2,size_infl_proot3,size_infl_proot5
0,109991185,40.053816,-75.240687,3,NaN,NaN,POINT (-75.24069 40.05382),POINT (-5697695.034 8483120.448),0,-5.697695e+06,...,1.0,1.0,"POLYGON ((-75.23914 40.05656, -75.23880 40.056...",198723.068185,4.192678e+06,0.047398,1.990313,1.410785,1.257884,1.147583
1,109991191,40.054803,-75.239081,3,NaN,NaN,POINT (-75.23908 40.05480),POINT (-5697474.612 8482974.522),0,-5.697475e+06,...,1.0,1.0,"POLYGON ((-75.24278 40.05572, -75.24265 40.055...",291442.444263,4.192678e+06,0.069512,2.918945,1.708492,1.429142,1.238925
2,109991199,40.055776,-75.237501,3,NaN,NaN,POINT (-75.23750 40.05578),POINT (-5697257.444 8482831.074),0,-5.697257e+06,...,1.0,1.0,"POLYGON ((-75.24077 40.05416, -75.24096 40.054...",245064.061026,4.192678e+06,0.058450,2.454442,1.566666,1.348914,1.196714
3,109991204,40.056232,-75.236783,3,NaN,NaN,POINT (-75.23678 40.05623),POINT (-5697156.554 8482766.601),0,-5.697157e+06,...,1.0,1.0,"POLYGON ((-75.24005 40.05461, -75.24024 40.054...",187511.352028,4.192678e+06,0.044724,1.878022,1.370409,1.233768,1.134332
4,110169862,40.063972,-75.249174,3,NaN,NaN,POINT (-75.24917 40.06397),POINT (-5696411.964 8484679.155),0,-5.696412e+06,...,1.0,1.0,"POLYGON ((-75.24676 40.06163, -75.24707 40.061...",194283.946127,4.192678e+06,0.046339,1.945853,1.394938,1.248447,1.142410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7315,3425873720,39.966878,-75.277524,3,NaN,NaN,POINT (-75.27752 39.96688),POINT (-5713004.185 8483178.234),471,-5.713004e+06,...,1.0,1.0,"POLYGON ((-75.27647 39.96684, -75.27650 39.966...",19734.719624,1.973472e+04,1.000000,0.197653,0.444582,0.582507,0.723071
7316,110006563,39.961942,-75.273003,3,NaN,NaN,POINT (-75.27300 39.96194),POINT (-5713613.274 8482370.035),472,-5.713613e+06,...,0.0,1.0,"POLYGON ((-75.27493 39.96162, -75.27515 39.961...",47176.323920,4.717632e+04,1.000000,0.472495,0.687383,0.778871,0.860755
7317,110133414,39.961600,-75.274141,3,NaN,NaN,POINT (-75.27414 39.96160),POINT (-5713712.736 8482492.281),472,-5.713713e+06,...,0.0,1.0,"POLYGON ((-75.27493 39.96162, -75.27515 39.961...",47176.323920,4.717632e+04,1.000000,0.472495,0.687383,0.778871,0.860755
7318,923789083,39.962719,-75.274196,3,NaN,NaN,POINT (-75.27420 39.96272),POINT (-5713536.406 8482556.428),472,-5.713536e+06,...,0.0,1.0,"POLYGON ((-75.27493 39.96162, -75.27515 39.961...",47176.323920,4.717632e+04,1.000000,0.472495,0.687383,0.778871,0.860755


In [8]:
# Block 5.5 (not in use, buffer is 0, thus retains all the park entry points as is)

# Get buffer of nodes close to each other.
ParkCombs = list([])
for i in range(len(cities)):
    
    # Get the buffer
    ParkComb = ParkRoads[i]
    ParkComb['geometry_m_buffer'] = ParkComb['geometry_m'].buffer(park_entry_point_merge)
    
    # Get and merge components
    M = libpysal.weights.fuzzy_contiguity(ParkComb['geometry_m_buffer'])
    ParkComb['components'] = M.component_labels
    
    # Take centroid of merged components
    centr = gpd.GeoDataFrame(ParkComb, geometry = 'geometry_x', crs = 4326).dissolve('components')['geometry_x'].centroid
    centr = gpd.GeoDataFrame(centr)
    centr.columns = ['comp_centroid']
    
    # Get node closest to the centroid of all merged nodes, which accesses the road network.
    ParkComb = pd.merge(ParkComb, centr, left_on = 'components', right_index = True)
    ParkComb['centr_dist'] = ParkComb['geometry_x'].distance(ParkComb['comp_centroid'])
    ParkComb = ParkComb.iloc[ParkComb.groupby('components')['centr_dist'].idxmin()]
    ParkCombs.append(ParkComb)
ParkCombs[0]

,osmid,y,x,street_count,highway,ref,geometry_x,geometry_m,Park_No,park_lon,...,park_area,share_walked,size_infl_factor,size_infl_proot2,size_infl_proot3,size_infl_proot5,geometry_m_buffer,components,comp_centroid,centr_dist
0,109991185,40.053816,-75.240687,3,NaN,NaN,POINT (-75.24069 40.05382),POINT (-5697695.034 8483120.448),0,-5.697695e+06,...,4.192678e+06,0.047398,1.990313,1.410785,1.257884,1.147583,POLYGON EMPTY,0,POINT (-75.24069 40.05382),0.0
1,109991191,40.054803,-75.239081,3,NaN,NaN,POINT (-75.23908 40.05480),POINT (-5697474.612 8482974.522),0,-5.697475e+06,...,4.192678e+06,0.069512,2.918945,1.708492,1.429142,1.238925,POLYGON EMPTY,1,POINT (-75.23908 40.05480),0.0
2,109991199,40.055776,-75.237501,3,NaN,NaN,POINT (-75.23750 40.05578),POINT (-5697257.444 8482831.074),0,-5.697257e+06,...,4.192678e+06,0.058450,2.454442,1.566666,1.348914,1.196714,POLYGON EMPTY,2,POINT (-75.23750 40.05578),0.0
3,109991204,40.056232,-75.236783,3,NaN,NaN,POINT (-75.23678 40.05623),POINT (-5697156.554 8482766.601),0,-5.697157e+06,...,4.192678e+06,0.044724,1.878022,1.370409,1.233768,1.134332,POLYGON EMPTY,3,POINT (-75.23678 40.05623),0.0
4,110169862,40.063972,-75.249174,3,NaN,NaN,POINT (-75.24917 40.06397),POINT (-5696411.964 8484679.155),0,-5.696412e+06,...,4.192678e+06,0.046339,1.945853,1.394938,1.248447,1.142410,POLYGON EMPTY,4,POINT (-75.24917 40.06397),0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7315,3425873720,39.966878,-75.277524,3,NaN,NaN,POINT (-75.27752 39.96688),POINT (-5713004.185 8483178.234),471,-5.713004e+06,...,1.973472e+04,1.000000,0.197653,0.444582,0.582507,0.723071,POLYGON EMPTY,7315,POINT (-75.27752 39.96688),0.0
7316,110006563,39.961942,-75.273003,3,NaN,NaN,POINT (-75.27300 39.96194),POINT (-5713613.274 8482370.035),472,-5.713613e+06,...,4.717632e+04,1.000000,0.472495,0.687383,0.778871,0.860755,POLYGON EMPTY,7316,POINT (-75.27300 39.96194),0.0
7317,110133414,39.961600,-75.274141,3,NaN,NaN,POINT (-75.27414 39.96160),POINT (-5713712.736 8482492.281),472,-5.713713e+06,...,4.717632e+04,1.000000,0.472495,0.687383,0.778871,0.860755,POLYGON EMPTY,7317,POINT (-75.27414 39.96160),0.0
7318,923789083,39.962719,-75.274196,3,NaN,NaN,POINT (-75.27420 39.96272),POINT (-5713536.406 8482556.428),472,-5.713536e+06,...,4.717632e+04,1.000000,0.472495,0.687383,0.778871,0.860755,POLYGON EMPTY,7318,POINT (-75.27420 39.96272),0.0


In [9]:
# Block 6 grid-parkentry combinations within euclidean threshold distance

start_time = time.time()
RoadComb = list()
for l in range(len(cities)):
    print(cities[l])
    # Check all parks within 1000m radius
    len1 = len(grids[l])
    len2 = len(ParkCombs[l])
    len3 = int(np.ceil(len2/1000))
    output = pd.DataFrame()
    len_mat = 0
    # Checking all the combinations at once is too performance intensive, it is broken down per 1000 (or what you want)
    for i in range(len3):
        # Check all grid-park combinations per 1000
        l1, l2 = range(0,len1), range(i*1000,(i+1)*1000)
        listed = pd.DataFrame(list(product(l1, l2)))

        # Merge grid and park information
        grid_merged = pd.merge(listed, 
                               grids[l][['grid_id','grid_lon','grid_lat','centroid','centroid_m']],
                               left_on = 0, right_index = True)
        node_merged = pd.merge(grid_merged, 
                               ParkCombs[l][['Park_No','osmid','geometry_x','geometry_y','geometry_m','park_lon','park_lat',
                                   'size_infl_proot2','size_infl_proot3','size_infl_proot5','share_walked','park_area','walk_area']], 
                               left_on = 1, right_index = True)

        # Preset index for merging
        node_merged['key'] = range(0,len(node_merged))
        node_merged = node_merged.set_index('key')
        node_merged = node_merged.loc[:, ~node_merged.columns.isin(['index'])]

        # Create lists for better computational performance
        glon = list(node_merged['grid_lon'])
        glat = list(node_merged['grid_lat'])
        plon = list(node_merged['park_lon'])
        plat = list(node_merged['park_lat'])
        infl2 = list(node_merged['size_infl_proot2'])
        infl3 = list(node_merged['size_infl_proot3'])
        infl5 = list(node_merged['size_infl_proot5'])

        # Get the euclidean distances
        mat = list()
        mat2 = list()
        mat3 = list()
        mat4 = list()
        for j in range(len(node_merged)):
            mat.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2))
            mat2.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl2[j])
            mat3.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl3[j])
            mat4.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl5[j])

        # Check if distances are within 1000m and join remaining info and concat in master df per 1000.
        mat_df = pd.DataFrame(mat3)[(np.array(mat) <= np.max(thresholds)) | 
                                    (np.array(mat2) <= np.max(thresholds)) | 
                                    (np.array(mat3) <= np.max(thresholds)) | 
                                    (np.array(mat4) <= np.max(thresholds))]

        # join the other gravity euclidean scores and other information
        mat_df = mat_df.join(pd.DataFrame(mat), lsuffix='_infl', rsuffix='_entr', how = 'left')
        mat_df = mat_df.join(pd.DataFrame(mat2), lsuffix='_entry', rsuffix='_pwr', how = 'left')
        mat_df = mat_df.join(pd.DataFrame(mat4), lsuffix='_pwr', rsuffix='_root', how = 'left')
        mat_df.columns = ['size_infl_eucl2','raw euclidean','size_infl_eucl3','size_infl_eucl5']    
        mat_df = mat_df.join(node_merged)

        output = pd.concat([output, mat_df])

        print((i+1),'/',len3,'comb. done',round((time.time() - start_time) / 60,2),' mns')
        print('of',np.where(i+1 == len3, len2 % 1000 * 1000, len1*1000) ,'within a Gravity model variant in one of',
              thresholds,'m threshold:',len(mat_df))

        # Checks the number of the parks within 1000m.
        len_mat = len_mat + len(mat_df)

    # Renaming columns
    print('total combinations within distance',len_mat)
    
    output.columns = ['size_infl_eucl3','raw euclidean','size_infl_eucl2','size_infl_eucl5',
                      'Grid_No','Park_entry_No','Grid_Id','grid_lon','grid_lat','Grid_coords_centroid','Grid_m_centroid',
                      'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid',
                      'park_lon','park_lat','size_infl_proot2','size_infl_proot3','size_infl_proot5',
                      'parkshare_walked','park_area','walk_area_m2']
    output = output[['raw euclidean','size_infl_eucl2','size_infl_eucl3','size_infl_eucl5',
                     'Grid_No','Park_entry_No','Grid_Id','Grid_coords_centroid','Grid_m_centroid',
                      'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid',
                     'walk_area_m2','size_infl_proot2','size_infl_proot3','size_infl_proot5']]
    
    # Reinstate geographic elements
    output = gpd.GeoDataFrame(output, geometry = 'Grid_coords_centroid', crs = 4326)
    output['Grid_m_centroid'] = gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043)
    output['Parkroad_coords_centroid'] = gpd.GeoSeries(output['Parkroad_coords_centroid'], crs = 4326)
    output['Parkroad_m_centroid'] = gpd.GeoSeries(output['Parkroad_m_centroid'], crs = 3043)
    
    # Get the nearest entrance point for the grid centroids
    mat5 = list()
    for i in range(len(output)):
        nearest = int(roads[l]['geometry'].sindex.nearest(output['Grid_coords_centroid'].iloc[i])[1])
        mat5.append(roads[l]['osmid'].iloc[nearest])
        if i % 250000 == 0: print(round(i/len(output)*100,1),'% gridentry done', round((time.time() - start_time) / 60,2),' mns')
            
    # format resulting dataframe
    output['grid_osm'] = mat5
    output = pd.merge(output, road_nodes[l]['geometry'], left_on = 'grid_osm', right_index = True)
    output['geometry_m'] = gpd.GeoSeries(output['geometry'], crs = 4326).to_crs(3043)
    output['grid_entry_dist'] = round(gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043
                                                   ).distance(output['geometry_m']),3)
    output = output.reset_index()
    print('100 % gridentry done', round((time.time() - start_time) / 60,2),' mns')
    RoadComb.append(output)
RoadComb[0]


Philadelphia, United States
1 / 8 comb. done 0.82  mns
of 7873000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 62057
2 / 8 comb. done 1.73  mns
of 7873000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 56770
3 / 8 comb. done 2.58  mns
of 7873000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 70708
4 / 8 comb. done 3.39  mns
of 7873000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 57345
5 / 8 comb. done 4.2  mns
of 7873000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 37151
6 / 8 comb. done 5.0  mns
of 7873000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 38448
7 / 8 comb. done 5.8  mns
of 7873000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 37858
8 / 8 comb. done 6.15  mns
of 320000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 9453
total combinations within distance 369790
0.0 % gridentry

,index,raw euclidean,size_infl_eucl2,size_infl_eucl3,size_infl_eucl5,Grid_No,Park_entry_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,...,Parkroad_coords_centroid,Parkroad_m_centroid,walk_area_m2,size_infl_proot2,size_infl_proot3,size_infl_proot5,grid_osm,geometry,geometry_m,grid_entry_dist
0,1798,1373.392587,973.495618,1091.828015,1196.769244,1798,0,6122,POINT (-75.24247 40.06191),POINT (-5696475.919 8483752.877),...,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185,1.410785,1.257884,1.147583,110562147,POINT (-75.24257 40.06216),POINT (-5696439.791 8483778.131),44.080
1,9671,1266.184486,741.112318,885.975442,1022.002141,1798,1,6122,POINT (-75.24247 40.06191),POINT (-5696475.919 8483752.877),...,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697474.612 8482974.522),291442.444263,1.708492,1.429142,1.238925,110562147,POINT (-75.24257 40.06216),POINT (-5696439.791 8483778.131),44.080
2,17544,1208.512478,771.391360,895.915175,1009.858654,1798,2,6122,POINT (-75.24247 40.06191),POINT (-5696475.919 8483752.877),...,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697257.444 8482831.074),245064.061026,1.566666,1.348914,1.196714,110562147,POINT (-75.24257 40.06216),POINT (-5696439.791 8483778.131),44.080
3,25417,1198.333947,874.434983,971.279681,1056.422706,1798,3,6122,POINT (-75.24247 40.06191),POINT (-5696475.919 8483752.877),...,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697156.554 8482766.601),187511.352028,1.370409,1.233768,1.134332,110562147,POINT (-75.24257 40.06216),POINT (-5696439.791 8483778.131),44.080
4,33290,928.483660,665.609100,743.711087,812.741169,1798,4,6122,POINT (-75.24247 40.06191),POINT (-5696475.919 8483752.877),...,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5696411.964 8484679.155),194283.946127,1.394938,1.248447,1.142410,110562147,POINT (-75.24257 40.06216),POINT (-5696439.791 8483778.131),44.080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369785,1176726,344.985923,300.173715,314.423910,326.309628,3649,7149,8621,POINT (-75.05157 40.03257),POINT (-5693569.140 8458902.388),...,"POLYGON ((-75.04925 40.02741, -75.04927 40.027...",POINT (-5693836.319 8458684.144),131881.633263,1.149288,1.097200,1.057235,5550924420,POINT (-75.05153 40.03264),POINT (-5693556.898 8458900.194),12.437
369786,1184599,346.522245,305.733278,318.766138,329.591313,3649,7150,8621,POINT (-75.05157 40.03257),POINT (-5693569.140 8458902.388),...,"POLYGON ((-75.04925 40.02741, -75.04927 40.027...",POINT (-5693748.199 8458605.714),128263.681386,1.133414,1.087074,1.051369,5550924420,POINT (-75.05153 40.03264),POINT (-5693556.898 8458900.194),12.437
369787,1192472,889.861657,794.136394,824.842249,850.259584,3649,7151,8621,POINT (-75.05157 40.03257),POINT (-5693569.140 8458902.388),...,"POLYGON ((-75.04925 40.02741, -75.04927 40.027...",POINT (-5694083.340 8458176.129),125366.550695,1.120540,1.078826,1.046576,5550924420,POINT (-75.05153 40.03264),POINT (-5693556.898 8458900.194),12.437
369788,2057899,1243.010494,983.090589,1063.050651,1131.676487,3046,7261,7826,POINT (-75.24247 40.04120),POINT (-5699773.963 8482691.420),...,"POLYGON ((-75.25235 40.04169, -75.25188 40.041...",POINT (-5700276.861 8483828.156),159620.769751,1.264391,1.169286,1.098380,5550942467,POINT (-75.24175 40.04045),POINT (-5699865.640 8482564.664),156.434


In [10]:
# Number of combinations within range per city
for i in range(len(RoadComb)): print(cities[i], len(RoadComb[i]))

Philadelphia, United States 369790
Denver, United States 402611
Ghent, Belgium 497547
Amsterdam, Netherlands 455510
Dhaka Metropolitan, Bangladesh 7868


In [49]:
# ADJUSTED Block 7 calculate route networks of all grid-parkentry combinations within euclidean threshold distance

warnings.filterwarnings("ignore")
start_time = time.time()

Routes = list()
for j in range(len(cities)):
    
    # Set variables
    Graph = graphs[j]
    CityRoads = RoadComb[j] # iloc to test the iteration speed.
        
    parknode = list(CityRoads['Parkroad_osmid'])
    gridnode = list(CityRoads['grid_osm'])
    Route_parts = pd.DataFrame()
    lengths = 0
    len1 = len(CityRoads)
    len2 = int(np.ceil(len(CityRoads)/250000))
    
    # Divide in chunks of 250000 for computational load
    for k in range(len2):
        
        CityRoad = CityRoads.iloc[250000*k:250000*k+250000]                
        s_mat = list([])
        s_mat1 = list([])
        s_mat2 = list([])
        s_mat3 = list([])
        s_mat4 = list([])
        print(cities[j].rsplit(',')[0],'chunk',k,'of',len2,':',
              k*250000,'-',k*250000+np.where(k*250000+250000 >= len(CityRoad),len(CityRoad),250000))
        for i in range(k*250000,k*250000+np.where(k*250000+250000 >= len(CityRoad),len(CityRoad),250000)):
            # First try from population grid to park Dijkstra algorithm in chunk
            try:
                shortest = nx.shortest_path(Graph, gridnode[i], parknode[i], 'travel_dist', method = 'dijkstra')
                s_mat.append(shortest)
                shortest_to = list(shortest[1:len(shortest)])
                shortest_to.append(0)
                s_mat1.append(shortest_to)
                s_mat2.append(list(np.repeat(i, len(shortest))))
                s_mat3.append(list(np.arange(0, len(shortest))))
                s_mat4.append('normal way')
            except:
                try:
                    # Check the reverse
                    shortest = nx.shortest_path(Graph, parknode[i], gridnode[i], 'travel_dist', method = 'dijkstra')
                    s_mat.append(shortest)
                    shortest_to = list(shortest[1:len(shortest)])
                    shortest_to.append(0)
                    s_mat1.append(shortest_to)
                    s_mat2.append(list(np.repeat(i, len(shortest))))
                    s_mat3.append(list(np.arange(0, len(shortest))))
                    s_mat4.append('reverse way')
                except:
                    # Otherwise the nearest node is taken, which is iterated 10 times at max. Order in route for nearest node:
                    # 1. gridnode to nearest to the original failed parknode
                    # 2. The reverse of 1.
                    # 3. nearest gridnode to the failed one and route to park
                    # 4. The reverse of 3.
                    len3 = 0
                    alt_route = list([])
                    while len3 < 10 and len(alt_route) < 1:

                        len3 = len3 +1
                        # Check which geom is equal to the grid osm
                        g_geom = gpd.GeoSeries(ParkRoads[0][ParkRoads[0]['osmid'] 
                                                        == int(RoadComb[0].iloc[1076863:1076863+1]['grid_osm'])]['geometry_x'])
                        
                        # Check distances from original to others.
                        g_nearest = pd.DataFrame((abs(float(g_geom.x) - gpd.GeoSeries(ParkRoads[0]['geometry_x']).x)**2
                        +abs(float(g_geom.y) - gpd.GeoSeries(ParkRoads[0]['geometry_x']).y)**2)**(1/2)
                                                ).join(ParkRoads[0]['osmid']).sort_values(0)

                        g_grid = g_nearest.iloc[len3,1]
                        g_park = RoadComb[0].iloc[1076863]['Parkroad_osmid']

                        p_geom = gpd.GeoSeries(ParkRoads[0][ParkRoads[0]['osmid'] 
                                                        == int(RoadComb[0].iloc[1076863:1076863+1]['Parkroad_osmid'])]['geometry_x'])
                        p_nearest = pd.DataFrame((abs(float(p_geom.x) - gpd.GeoSeries(ParkRoads[0]['geometry_x']).x)**2
                            +abs(float(p_geom.y) - gpd.GeoSeries(ParkRoads[0]['geometry_x']).y)**2)**(1/2)
                                                ).join(ParkRoads[0]['osmid']).sort_values(0)

                        p_grid = RoadComb[0].iloc[1076863]['grid_osm']
                        p_park = p_nearest.iloc[len3,1]

                        print('original grid',p_grid, 'original park',g_park, 'alt park', p_park, 'alt grid', g_grid)
                        try:
                            alt_route.append(nx.shortest_path(graphs[0], p_grid, p_park, 'travel_dist', method = 'dijkstra'))
                        except:
                            try:
                                alt_route.append(nx.shortest_path(graphs[0], p_park, p_grid, 'travel_dist', method = 'dijkstra'))
                            except:
                                try:
                                    alt_route.append(nx.shortest_path(graphs[0], g_grid, g_park, 'travel_dist', method = 'dijkstra'))
                                except:
                                    try:
                                        alt_route.append(nx.shortest_path(graphs[0], g_grid, g_park, 'travel_dist', method = 'dijkstra'))
                                    except:
                                        print('no route between grid and park and in all comb. of 10 nearest park and grids osmids')
                                        pass
                        
                        s_mat.append(alt_route[0])
                        shortest_to = list(shortest[1:len(shortest)])
                        shortest_to.append(0)
                        s_mat1.append(shortest_to)
                        s_mat2.append(list(np.repeat(i, len(shortest))))
                        s_mat3.append(list(np.arange(0, len(shortest))))
                        s_mat4.append('normal way')

            if i+1 % 25000 == 0: print('         ',round(i/len(CityRoads)*100,2),'% done', 
                                     round((time.time() - start_time) / 60,2),'mns')
                
        print('          ',round(i/len(CityRoads)*100,2),'% pathfinding done', round((time.time() - start_time) / 60,2),' mns')

        # Unpack lists
        s_mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat) for i in b]
        s_mat_u1 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat1) for i in b]
        s_mat_u2 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat2) for i in b]
        s_mat_u3 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat3) for i in b]

        # Format df
        routes = pd.DataFrame([s_mat_u,s_mat_u1,s_mat_u2,s_mat_u3]).transpose()
        routes.columns = ['from','to','route','step']
        mat_key = list([])
        for i in range(len(routes)):
            mat_key.append(str(int(s_mat_u[i])) + '-' + str(int(s_mat_u1[i])))
        routes['key'] = mat_key
        routes = routes.set_index('key')

        # Add route information
        routes = routes.join(road_conn[j], how = 'left')
        routes = gpd.GeoDataFrame(routes, geometry = 'geometry', crs = 4326)
        routes.sort_values(by = ['route','step'])
        print('          formatting done', round((time.time() - start_time) / 60,2), 'mns')

        # get single (dissolved) line per route, attach information.
        routes2 = routes[['route','geometry']].dissolve('route')
        routes2['way_calculated'] = s_mat4
        routes2['route_cost'] = routes.groupby('route')['length'].sum()
        routes2['num_steps'] = routes.groupby('route')['step'].max()
        routes2.index = routes2.index.astype(int)
        routes2 = pd.merge(routes2, CityRoads[['Grid_No','Park_No','Park_entry_No','grid_entry_dist','Parkroad_osmid','grid_osm',
                                              'walk_area_m2','size_infl_proot2','size_infl_proot3', 'size_infl_proot5',
                                              'raw euclidean']],
                           left_index = True, right_index = True)
        routes2['raw_total_cost'] = routes2['route_cost'] + routes2['grid_entry_dist']
        routes2['grav2_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_proot2']
        routes2['grav3_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_proot3']
        routes2['grav5_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_proot5']

        routes2['gridpark_no'] = routes2['Grid_No'].astype(str) +'-'+ routes2['Park_No'].astype(str)

        print('          dissolving done', round((time.time() - start_time) / 60,2), 'mns')
        Route_parts = pd.concat([Route_parts, routes2])
    #Route_parts = Route_parts.reset_index()
    Routes.append(Route_parts)
Routes[0]                



Philadelphia chunk 0 - 250000
          0.0 % done 0.0 mns
          2.7 % done 0.12 mns
          5.41 % done 0.29 mns
          8.11 % done 0.48 mns
          10.82 % done 0.75 mns
          13.52 % done 1.09 mns
          16.23 % done 1.4 mns
          18.93 % done 1.6 mns
          21.63 % done 1.78 mns
          24.34 % done 2.02 mns
          27.04 % done 2.22 mns
          29.75 % done 2.4 mns
          32.45 % done 2.61 mns
          35.16 % done 2.93 mns
          37.86 % done 3.14 mns
          40.56 % done 3.35 mns
          43.27 % done 3.54 mns
          45.97 % done 3.75 mns
          48.68 % done 3.93 mns
          51.38 % done 4.12 mns
          54.08 % done 4.35 mns
          56.79 % done 4.62 mns
          59.49 % done 4.89 mns
          62.2 % done 5.28 mns
          64.9 % done 5.55 mns
          100 % pathfinding done 5.77  mns
          formatting done 7.71 mns
          dissolving done 8.81 mns
Philadelphia chunk 250000 - 369790
          67.61 % done 8.81 mns
  

,index,geometry,way_calculated,route_cost,num_steps,Grid_No,Park_No,Park_entry_No,grid_entry_dist,Parkroad_osmid,...,walk_area_m2,size_infl_proot2,size_infl_proot3,size_infl_proot5,raw euclidean,raw_total_cost,grav2_total_cost,grav3_total_cost,grav5_total_cost,gridpark_no
0,0,"MULTILINESTRING ((-75.24344 40.05613, -75.2426...",normal way,1726.207,11,1798,0,0,44.080,109991185,...,198723.068185,1.410785,1.257884,1.147583,1373.392587,1770.287,1254.824479,1407.353556,1542.621575,1798-0
1,1,"MULTILINESTRING ((-75.24069 40.05382, -75.2390...",normal way,1901.505,12,1798,0,1,44.080,109991191,...,291442.444263,1.708492,1.429142,1.238925,1266.184486,1945.585,1138.773240,1361.366016,1570.380981,1798-0
2,2,"MULTILINESTRING ((-75.24069 40.05382, -75.2390...",normal way,2074.101,13,1798,0,2,44.080,109991199,...,245064.061026,1.566666,1.348914,1.196714,1208.512478,2118.181,1352.031155,1570.286229,1769.996960,1798-0
3,3,"MULTILINESTRING ((-75.24053 40.06186, -75.2402...",normal way,1456.034,13,1798,0,3,44.080,109991204,...,187511.352028,1.370409,1.233768,1.134332,1198.333947,1500.114,1094.646583,1215.879973,1322.464823,1798-0
4,4,"MULTILINESTRING ((-75.24643 40.06222, -75.2468...",normal way,782.483,6,1798,0,4,44.080,110169862,...,194283.946127,1.394938,1.248447,1.142410,928.483660,826.563,592.544467,662.073113,723.525689,1798-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369785,369785,"MULTILINESTRING ((-75.05035 40.03162, -75.0502...",normal way,314.046,6,3649,447,7149,12.437,5550924378,...,131881.633263,1.149288,1.097200,1.057235,344.985923,326.483,284.074243,297.560146,308.808387,3649-447
369786,369786,"MULTILINESTRING ((-75.05035 40.03162, -75.0502...",normal way,235.827,5,3649,447,7150,12.437,5550924379,...,128263.681386,1.133414,1.087074,1.051369,346.522245,248.264,219.040964,228.378287,236.133925,3649-447
369787,369787,"MULTILINESTRING ((-75.05035 40.03162, -75.0502...",normal way,597.711,12,3649,447,7151,12.437,5854222462,...,125366.550695,1.120540,1.078826,1.046576,889.861657,610.148,544.512429,565.566394,582.994200,3649-447
369788,369788,"MULTILINESTRING ((-75.24770 40.04671, -75.2477...",normal way,7175.780,18,3046,462,7261,156.434,945875753,...,159620.769751,1.264391,1.169286,1.098380,1243.010494,7332.214,5799.010239,6270.675027,6675.482003,3046-462


In [69]:
# Block 8 determine best parkentry points from each grid, then calculate grid scores
# and finally aggregate city access in categories (high, medium, low and no access)

start_time = time.time()
popg_acc = pd.DataFrame()
adj_popg_acc = pd.DataFrame()
grid_scores = list([])
gridpark = list([])
for n in range(len(cities)):    
    print(cities[n])
    
    # For the four distance decay variants regarding park size.
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    grid_score = list([])
    gridparks = list([])
    gridpark.append(gridparks)
    popgrid_access = pd.DataFrame()
    adj_popgrid_access = pd.DataFrame()
    for i in range(len(l1)):
        # Get the lowest indices grouped by a key consisting of grid no and park no (best entry point from a grid to a park)
        str1 = 'gridpark_' + l1[i]
        locals()[str1] = Routes[n].iloc[Routes[n].groupby('gridpark_no')[(str(l1[i]) +'_total_cost')].idxmin()]
        l2 = list()
        
        # Get the total cost column
        for j in enumerate(l1): 
            if j[0] != i: l2.append(j[1] + '_total_cost')
        locals()[str1] = locals()[str1].loc[:, ~locals()[str1].columns.isin(l2)]
        
        # Get grid information
        locals()[str1] = pd.merge(locals()[str1], grids[n][['PoP2015_Number','geometry']],
                                left_on = 'Grid_No', right_index = True, how = 'outer')
        locals()[str1] = locals()[str1].reset_index()
        
        # formatting
        locals()[str1]['Park_No'] = locals()[str1]['Park_No'].fillna(-1)
        locals()[str1]['Park_No'] = locals()[str1]['Park_No'].astype(int)
        locals()[str1]['Park_entry_No'] = locals()[str1]['Park_entry_No'].fillna(-1)
        locals()[str1]['Park_entry_No'] = locals()[str1]['Park_entry_No'].astype(int)
        
        grdsc = pd.DataFrame()
        gridsc = pd.DataFrame()
        print(m1[i], round((time.time() - start_time) / 60,2), 'mns')
        
        # For each threshold given, calculate a score
        for k in range(len(thresholds)):
            t = thresholds[k]
            str2 = str(t)
            score = 'tr_'+ str2
            adj_score = 'adj_tr_' + str2
            
            #Only get routes within the threshold given (it loops over every threshold) and calculate the scores
            thold = locals()[str1][locals()[str1][l1[i] + '_total_cost'] <= t]
            thold[score] = t - thold[l1[i] + '_total_cost']
            thold['pop' + score] = thold[score] * thold['PoP2015_Number']
            thold[adj_score] = thold[score] * ((math.pi*600**2) / (math.pi*t**2)) / (600/t)
            thold['walk_area_ha' + str2] = locals()[str1]['walk_area_m2'] /10000
            thold['walkha_person' + str2] = thold['PoP2015_Number'] / thold['walk_area_ha' + str2]
            #thold = thold[thold[score] > 0]
            
            # Join the gridpark information from before.
            locals()[str1] = locals()[str1].join(thold[[score,'pop' + score,adj_score,'walk_area_ha' + str2, 'walkha_person' + str2]])
            # get the grid_scores
            gs = pd.DataFrame()
            gs[[score,'pop_' + score,adj_score,'walkha_' + score,'walkha_person_' + score]] = locals()[str1].groupby(
                    'Grid_No')[score,'pop' + score, adj_score, 'walk_area_ha' + str2, 'walkha_person' + str2].sum()
            
            trstr = locals()[str1][locals()[str1][score] > 0]
            gs[score + '_parks'] = trstr.groupby('Grid_No')['gridpark_no'].count()
            
            # Add the routes as a dissolved line_geom
            gs[score + '_routes'] = gpd.GeoDataFrame(trstr[['Grid_No','geometry_x']],
                                                          geometry = 'geometry_x', crs = 4326).dissolve('Grid_No')

            # Add parks which grids have access to with its closest access point
            gs[score+'Park:entry'] = trstr[trstr['Park_No'] >=0].groupby('Grid_No')['Park_No'].apply(list).astype(str
            ) + ':' + trstr[trstr['Park_entry_No'] >=0].groupby('Grid_No')['Park_entry_No'].apply(list).astype(str)
            
            # determine the thresholds category-score. 
            # High >= threshold (perfect score to one park), medium is above half perfect, 
            # low is below this and no is no access to a park for a certain grid within the threshold given
            gs[score+'_access'] = np.select([gs[score] >= t, (gs[score] < t) & (
            gs[score]>= t/2), (gs[score] < t/2) & (gs[score]> 0), gs[score] <= 0],
                  ['1 high','2 medium','3 low','4 no'])
                        
            gs[score+'_adj_access'] = np.select([gs[adj_score] >= t, (gs[adj_score] < t) & (
            gs[adj_score]>= t/2), (gs[adj_score] < t/2) & (gs[adj_score]> 0), gs[adj_score] <= 0],
                  ['1 high','2 medium','3 low','4 no'])
            
            gs = gs.join(grids[n]['PoP2015_Number'], how = 'outer')
            grdsc = pd.concat([grdsc, gs], axis = 1)
            
            gs = gpd.GeoDataFrame(gs, geometry = score + '_routes', crs = 4326)
            gs.to_file('D:Dumps/Scores output/Grid_lines_shp/gridscore_'+ l1[i] + '_' + str2 + '_' + cities[n] + '.shp')
            
            gsc = gs.loc[:,~gs.columns.isin([score + '_routes'])]
            gridsc = pd.concat([gridsc, gsc])

            # Group according to the categories just created and sum the populations living in those grids
            popgacc = pd.DataFrame()
            popgacc[m1[i]+'_'+str(t)] = gs.groupby(score+'_access')['PoP2015_Number'].sum()
            popgrid_access = pd.concat([popgrid_access, popgacc],axis=1)   
            
            adj_popgacc = pd.DataFrame()
            adj_popgacc[m1[i]+'_'+str(t)] = gs.groupby(score+'_adj_access')['PoP2015_Number'].sum()
            adj_popgrid_access = pd.concat([adj_popgrid_access, adj_popgacc],axis=1)   
            print('grid ',t)
        
        grid_score.append(grdsc)
        
        gridsc = gridsc.join(grids[n]['geometry'])
        gridsc = gpd.GeoDataFrame(gridsc, geometry = 'geometry', crs = 4326)
        gridsc.to_file('D:Dumps/Scores output/Grid_geoms_shp/gridscore_'+ l1[i] + '_' + cities[n] + '.shp')
        
        # Detailed scores to files number of cities * ways to measure = number of files.
        # Different threshold-scores are in the same dataframe
        gridsc = gridsc.loc[:, gridsc.columns!='geometry']
        gridsc.to_csv('D:/Dumps/Scores output/Grid_csv/gridscore_'+ l1[i] + '_' + cities[n] + '.csv')
        gridparks.append(locals()[str1])
        
    grid_scores.append(grid_score)

    # For each city, divide the population access by group by the total to get its share.
    popgrid_access = popgrid_access / popgrid_access.sum()
    popgrid_access = pd.DataFrame(popgrid_access.unstack())
    popg_acc = pd.concat([popg_acc, popgrid_access], axis = 1)
    
    adj_popgrid_access = adj_popgrid_access / adj_popgrid_access.sum()
    adj_popgrid_access = pd.DataFrame(adj_popgrid_access.unstack())
    adj_popg_acc = pd.concat([adj_popg_acc, adj_popgrid_access], axis = 1)
    
    print(cities[n],'done', round((time.time() - start_time) / 60,2), 'mns')
popg_acc.columns = cities
adj_popg_acc.columns = cities

popg_acc.to_csv('D:/Dumps/Scores output/popgrid_access.csv')
adj_popg_acc.to_csv('D:/Dumps/Scores output/radius-euclidean adjusted popgrid access.csv')

adj_popg_acc    


Philadelphia, United States
entrance 0.03 mns
grid  300
grid  600
grid  1000
gravity**(1/2) 0.55 mns
grid  300
grid  600
grid  1000
gravity**(1/3) 0.94 mns
grid  300
grid  600
grid  1000
gravity**(1/5) 1.31 mns
grid  300
grid  600
grid  1000
Philadelphia, United States done 1.67 mns
Denver, United States
entrance 1.73 mns
grid  300
grid  600
grid  1000
gravity**(1/2) 2.16 mns
grid  300
grid  600
grid  1000
gravity**(1/3) 2.55 mns
grid  300
grid  600
grid  1000
gravity**(1/5) 2.9 mns
grid  300
grid  600
grid  1000
Denver, United States done 3.22 mns
Ghent, Belgium
entrance 3.26 mns
grid  300
grid  600
grid  1000
gravity**(1/2) 3.84 mns
grid  300
grid  600
grid  1000
gravity**(1/3) 4.24 mns
grid  300
grid  600
grid  1000
gravity**(1/5) 4.52 mns
grid  300
grid  600
grid  1000
Ghent, Belgium done 4.76 mns
Amsterdam, Netherlands
entrance 4.78 mns
grid  300
grid  600
grid  1000
gravity**(1/2) 5.24 mns
grid  300
grid  600
grid  1000
gravity**(1/3) 5.57 mns
grid  300
grid  600
grid  1000
gravi

Philadelphia, United States  \
entrance_300        1 high                       0.168534   
                    2 medium                     0.088019   
                    3 low                        0.118207   
                    4 no                         0.625240   
entrance_600        1 high                       0.135443   
                    2 medium                     0.310209   
                    3 low                        0.314918   
                    4 no                         0.239429   
entrance_1000       1 high                       0.212041   
                    2 medium                     0.385987   
                    3 low                        0.336423   
                    4 no                         0.065549   
gravity**(1/2)_300  1 high                       0.145584   
                    2 medium                     0.079893   
                    3 low                        0.086041   
                    4 no                         0.688482   
gravity**(1/2)_600  1 high                       0.043070   
                    2 medium                     0.294260   
                    3 low                        0.314254   
                    4 no                         0.348416   
gravity**(1/2)_1000 1 high                       0.032721   
                    2 medium                     0.357501   
                    3 low                        0.498418   
                    4 no                         0.111360   
gravity**(1/3)_300  1 high                       0.144581   
                    2 medium                     0.077893   
                    3 low                        0.086678   
                    4 no                         0.690848   
gravity**(1/3)_600  1 high                       0.049251   
                    2 medium                     0.287981   
                    3 low                        0.319271   
                    4 no                         0.343497   
gravity**(1/3)_1000 1 high                       0.041844   
                    2 medium                     0.364250   
                    3 low                        0.501135   
                    4 no                         0.092770   
gravity**(1/5)_300  1 high                       0.147036   
                    2 medium                     0.081131   
                    3 low                        0.091995   
                    4 no                         0.679838   
gravity**(1/5)_600  1 high                       0.063127   
                    2 medium                     0.293895   
                    3 low                        0.332417   
                    4 no                         0.310562   
gravity**(1/5)_1000 1 high                       0.073474   
                    2 medium                     0.386418   
                    3 low                        0.462087   
                    4 no                         0.078020   

                              Denver, United States  Ghent, Belgium  \
entrance_300        1 high                 0.171465        0.121734   
                    2 medium               0.090560        0.056590   
                    3 low                  0.109516        0.064218   
                    4 no                   0.628459        0.757457   
entrance_600        1 high                 0.116215        0.090736   
                    2 medium               0.316909        0.181750   
                    3 low                  0.329984        0.171527   
                    4 no                   0.236892        0.555986   
entrance_1000       1 high                 0.144736        0.132825   
                    2 medium               0.413410        0.213761   
                    3 low                  0.387950        0.246968   
                    4 no                   0.053903        0.406447   
gravity**(1/2)_300  1 high                 0.183522        0.135815   
                    2 medium               0.0

In [25]:
# Block 9 calculte park scores from previously determined best grid-park routes.

start_time = time.time()
cityparks = list([])
for i in range(len(cities)):
    
    # For the four distance decay variants regarding park size.
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    parks = list([])
    for j in range(len(l1)):
        parksc = pd.DataFrame()
        prksc = pd.DataFrame()
        for k in range(len(thresholds)):
            score = 'tr_' + str(thresholds[k])
            str2 = str(thresholds[k])
            str1 = gridpark[i][j][gridpark[i][j][score] > 0]

            # Get the park scores
            prk = pd.DataFrame()
            prk[[score,'pop_' + score,'walkha_' + score]] = str1.groupby(
                'Park_No')[score,'pop' + score,'walk_area_ha' + str2].sum()
            prk[score + '_parks'] = str1.groupby('Park_No')['gridpark_no'].count()
            
            # Add the routes as a dissolved line_geom
            prk[score+'route'] = gpd.GeoDataFrame(str1[['Park_No','geometry_x']], 
                             geometry = 'geometry_x', crs = 4326).dissolve('Park_No')
            
            # Add parks which grids have access to with its closest access point
            prk[score+'Grid:Pentry'] = str1[str1['Grid_No'] >=0].groupby('Park_No')['Grid_No'].apply(list).astype(str
            ) + ':' + str1[str1['Park_entry_No'] >=0].groupby('Park_No')['Park_entry_No'].apply(list).astype(str)
            
            # Get all parks, even with no score.
            prk = prk.join(parks_in_range[i].iloc[:,0], how = 'outer')
            prk = prk.loc[:,~prk.columns.isin(['components'])]
            #prk = prk.fillna(-1)
            print('park', thresholds[k])
            
            # Get the park score categories (same as grid score)
            prk[score+'_access'] = np.select([prk[score] >= t, (prk[score] < t) & (
                prk[score]>= t/2), (prk[score] < t/2) & (prk[score]> 0), prk[score] <= 0 | prk[score].isna()],
                ['1 high','2 medium','3 low','4 no'])

            parksc = pd.concat([parksc, prk], axis = 1)
                
            prk = gpd.GeoDataFrame(prk, geometry = score+'route', crs = 4326)
            prk.to_file('D:Dumps/Scores output/Park_lines_shp/parkscore_'+ l1[j] + '_' + str2 + '_' + cities[i] + '.shp')
            
            psc = prk.loc[:,~prk.columns.isin([score + 'route'])]
            prksc = pd.concat([prksc, psc])
            
        parks.append(parksc)
        
        prksc = prksc.join(parks_in_range[i]['park_geom'])
        prksc = gpd.GeoDataFrame(prksc, geometry = 'park_geom', crs = 4326)
        prksc.to_file('D:Dumps/Scores output/Grid_geoms_shp/parkscore_'+ l1[j] + '_' + cities[i] + '.shp')
        
        # Detailed scores to files number of cities * ways to measure = number of files.
        # Different threshold-scores are in the same dataframe
        prksc = prksc.loc[:, prksc.columns!='park_geom']
        prksc.to_csv('D:/Dumps/Scores output/Park_csv/parkscore_'+ l1[j] + '_' + cities[i]+ '.csv')

        print(m1[j], round((time.time() - start_time) / 60,2), 'mns')
    cityparks.append(parks)
    print(cities[i],'done', round((time.time() - start_time) / 60,2), 'mns')
pd.DataFrame(cityparks[1][1])

park 300
park 600
park 1000
entrance 0.19 mns
park 300
park 600
park 1000
gravity**(1/2) 0.5 mns
park 300
park 600
park 1000
gravity**(1/3) 0.73 mns
park 300
park 600
park 1000
gravity**(1/5) 0.9 mns
Philadelphia, United States done 0.9 mns
park 300
park 600
park 1000
entrance 1.0 mns
park 300
park 600
park 1000
gravity**(1/2) 1.15 mns
park 300
park 600
park 1000
gravity**(1/3) 1.26 mns
park 300
park 600
park 1000
gravity**(1/5) 1.35 mns
Denver, United States done 1.36 mns
park 300
park 600
park 1000
entrance 1.42 mns
park 300
park 600
park 1000
gravity**(1/2) 1.88 mns
park 300
park 600
park 1000
gravity**(1/3) 1.98 mns
park 300
park 600
park 1000
gravity**(1/5) 2.04 mns
Ghent, Belgium done 2.04 mns
park 300
park 600
park 1000
entrance 2.1 mns
park 300
park 600
park 1000
gravity**(1/2) 2.18 mns
park 300
park 600
park 1000
gravity**(1/3) 2.24 mns
park 300
park 600
park 1000
gravity**(1/5) 2.29 mns
Amsterdam, Netherlands done 2.29 mns
park 300
park 600
park 1000
entrance 2.31 mns
park 30

,tr_300,pop_tr_300,walkha_tr_300,tr_300_parks,tr_300route,tr_300Grid:Pentry,tr_300_access,tr_600,pop_tr_600,walkha_tr_600,...,tr_600route,tr_600Grid:Pentry,tr_600_access,tr_1000,pop_tr_1000,walkha_tr_1000,tr_1000_parks,tr_1000route,tr_1000Grid:Pentry,tr_1000_access
0,5152.901342,718127.749950,828.478500,34.0,"(LINESTRING (-104.8969235 39.6494712, -104.897...","[5875, 5907, 5937, 5938, 5939, 5969, 5970, 597...",1 high,20286.807075,3.043878e+06,1623.246975,...,"(LINESTRING (-104.8990746 39.6494014, -104.898...","[5874, 5875, 5876, 5906, 5907, 5908, 5909, 591...",1 high,61222.944653,9.404874e+06,3636.292392,136.0,"(LINESTRING (-104.8988304 39.6542774, -104.898...","[5765, 5766, 5767, 5805, 5806, 5807, 5808, 580...",1 high
1,339.648844,3431.447231,34.662520,2.0,"LINESTRING (-104.8940981 39.7728468, -104.8939...","[1123, 1124]:[49, 49]",3 low,1841.485172,1.792810e+04,173.312602,...,"(LINESTRING (-104.8940981 39.7728468, -104.893...","[1020, 1123, 1124, 1213, 1214, 1215, 1216, 130...",1 high,9576.302683,1.187309e+05,571.931586,33.0,"(LINESTRING (-104.8940981 39.7728468, -104.893...","[1019, 1020, 1021, 1022, 1119, 1123, 1124, 121...",1 high
2,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,...,NaN,NaN,0,2750.194500,3.497849e+05,1128.916403,20.0,"(LINESTRING (-105.0347238 39.7999519, -105.034...","[162, 211, 212, 213, 214, 215, 216, 217, 218, ...",1 high
3,5081.416908,927494.957763,1325.582206,33.0,"(LINESTRING (-104.8664599 39.6570882, -104.866...","[5602, 5660, 5661, 5662, 5663, 5664, 5665, 566...",1 high,19915.883172,3.241268e+06,2755.606551,...,"(LINESTRING (-104.866489 39.6604199, -104.8664...","[5415, 5479, 5480, 5481, 5542, 5544, 5545, 554...",1 high,66347.375419,9.709786e+06,5744.374924,149.0,"(LINESTRING (-104.8701508 39.6675265, -104.869...","[5246, 5287, 5289, 5350, 5351, 5352, 5413, 541...",1 high
4,11.006268,1177.670647,32.792558,1.0,"LINESTRING (-104.7946853 39.7982924, -104.7943...",[57]:[245],3 low,1173.334714,1.514246e+05,327.925582,...,"(LINESTRING (-104.796298 39.7955482, -104.7957...","[100, 144, 145, 192, 54, 55, 56, 57, 98, 99]:[...",1 high,8114.521400,9.925202e+05,852.606513,26.0,"(LINESTRING (-104.796298 39.7955482, -104.7957...","[100, 141, 142, 143, 144, 145, 189, 190, 191, ...",1 high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,...,NaN,NaN,0,140.016716,2.309741e+04,7.535758,2.0,"(LINESTRING (-104.9269664 39.7065548, -104.927...","[3905, 4108]:[7933, 7934]",3 low
337,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0
338,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0
339,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,...,NaN,NaN,0,363.091862,6.923236e+04,8.423828,2.0,"(LINESTRING (-105.0545596 39.7345148, -105.056...","[2705, 2706]:[7970, 7970]",3 low


In [27]:
grid_scores[0][0][grid_scores[0][0]['tr_300']>0]

,tr_300,pop_tr_300,walkha_tr_300,walkha_person_tr_300,tr_300_parks,tr_300_routes,tr_300Park:entry,tr_300_access,PoP2015_Number,tr_600,...,PoP2015_Number,tr_1000,pop_tr_1000,walkha_tr_1000,walkha_person_tr_1000,tr_1000_parks,tr_1000_routes,tr_1000Park:entry,tr_1000_access,PoP2015_Number
Grid_No,,,,,,,,,,,,,,,,,,,,,
44,34.583,4737.871,9.626691,14.231266,1.0,"(LINESTRING (-75.008283 40.125077, -75.007804 ...",[349]:[6080],3 low,137,334.583,...,137,734.583,100637.871,9.626691,14.231266,1.0,"(LINESTRING (-75.008283 40.125077, -75.007804 ...",[349]:[6080],2 medium,137
50,229.655,93699.240,11.479216,35.542496,1.0,None,[62]:[3178],2 medium,408,529.655,...,408,929.655,379299.240,11.479216,35.542496,1.0,None,[62]:[3178],2 medium,408
61,241.229,25087.816,9.626691,10.803297,1.0,None,[349]:[6080],2 medium,104,541.229,...,104,941.229,97887.816,9.626691,10.803297,1.0,None,[349]:[6080],2 medium,104
62,56.206,6070.248,9.638221,11.205387,1.0,"(LINESTRING (-75.004846 40.123052, -75.00514 4...",[349]:[6081],3 low,108,356.206,...,108,756.206,81670.248,9.638221,11.205387,1.0,"(LINESTRING (-75.004846 40.123052, -75.00514 4...",[349]:[6081],2 medium,108
79,246.378,51246.624,9.756757,21.318559,1.0,None,[349]:[6079],2 medium,208,546.378,...,208,946.378,196846.624,9.756757,21.318559,1.0,None,[349]:[6079],2 medium,208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7821,166.877,6841.957,20.854677,1.965986,1.0,"(LINESTRING (-75.2546916 39.8948727, -75.25471...",[10]:[583],2 medium,41,466.877,...,41,945.070,38747.870,21.257253,103.809924,2.0,"(LINESTRING (-75.2546916 39.8948727, -75.25471...","[10, 251]:[583, 5216]",2 medium,41
7824,150.987,6794.415,0.402577,111.779932,1.0,"(LINESTRING (-75.247101 39.8940713, -75.247127...",[251]:[5216],2 medium,45,648.472,...,45,1450.693,65281.185,29.996662,119.086811,3.0,"(LINESTRING (-75.250584 39.896673, -75.2515468...","[10, 251, 419]:[583, 5216, 6819]",1 high,45
7830,229.878,229.878,25.134823,0.039785,1.0,"(LINESTRING (-75.2558257 39.8928024, -75.25580...",[10]:[587],2 medium,1,529.878,...,1,929.878,929.878,25.134823,0.039785,1.0,"(LINESTRING (-75.2558257 39.8928024, -75.25580...",[10]:[587],2 medium,1


In [35]:
# Block 10 get the preferenced parks for each grid (lowest score) for all distance decay variants.
preference = list([])
for n in enumerate(cities): 
    print(n[1])
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    prefer = list([])
    for j in enumerate(l1):
        pref = list([])
        print(m1[j[0]])
        for k in thresholds:
            score = 'tr_'+ str(k)
            g = gridpark[n[0]][j[0]].iloc[gridpark[n[0]][j[0]].groupby('Grid_No')[score].idxmax().dropna().astype(int)]
            g = g[g[score] > 0]
            g.join(grids[n[0]]['grid_id'], how = 'outer')
            g.loc[:, (g.columns!='geometry_x') & (g.columns!='geometry_y')].to_csv('D:/Dumps/Scores output/Grid_pref_parks_csv/park-pref_' + j[1] +'-'+ str(k) + '-' + n[1] +'.csv')
            g_lines = gpd.GeoDataFrame(g.loc[:, ~g.columns.isin(['geometry_y'])], geometry = 'geometry_x', crs = 4326)
            g_lines.to_file('D:Dumps/Scores output/Grid_pref_parks_lines/park-pref_' +'-'+ j[1] + str(k) + '-' + n[1] +'.shp')
            g_geoms = gpd.GeoDataFrame(g.loc[:, ~g.columns.isin(['geometry_x'])], geometry = 'geometry_y', crs = 4326)
            g_geoms.to_file('D:Dumps/Scores output/Grid_pref_parks_geoms/park-pref_' +'-'+ j[1] + str(k) + '-' + n[1] +'.shp')
            pref.append(g)
            print('park_prefer',k)
        prefer.append(pref)
        len(pref)
    preference.append(prefer)
print('all done')

Philadelphia, United States
entrance
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/2)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/3)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/5)
park_prefer 300
park_prefer 600
park_prefer 1000
Denver, United States
entrance
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/2)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/3)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/5)
park_prefer 300
park_prefer 600
park_prefer 1000
Ghent, Belgium
entrance
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/2)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/3)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/5)
park_prefer 300
park_prefer 600
park_prefer 1000
Amsterdam, Netherlands
entrance
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/2)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/3)
park_prefer 300
park_pref

In [30]:
print(round((time.time() - start) / 60,2),'mns')

182.56 mns
